In [1]:
#default_exp haystack_code_search

In [2]:
#export
import pprint
import numpy as np
import pandas as pd
import requests
import torch

from haystack import Finder
from haystack.database.elasticsearch import ElasticsearchDocumentStore
from haystack.database.memory import InMemoryDocumentStore

from haystack.retriever.dense import EmbeddingRetriever
from haystack.utils import print_answers
from pytorch_hackathon import zero_shot_learning

In [3]:
%cd ..

/home/kuba/Projects/pytorch_hackathon


In [4]:
!ls data

feeds.txt  topics.txt  zsl_feed_results.csv


In [5]:
rss_feed_urls = list(pd.read_table('data/feeds.txt', header=None).iloc[:,0].values)

In [6]:
feed_df = zero_shot_learning.get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:08<00:00,  1.79it/s]
/home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/zero_shot_learning.py:69: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 69 of the file /home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/zero_shot_learning.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)


In [7]:
#export

use_gpu = torch.cuda.is_available()

In [8]:
pretty_print = pprint.PrettyPrinter(indent=2).pprint

In [9]:
feed_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,published_parsed,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail
0,I2L-MeshNet: Image-to-Lixel Prediction Network for Accurate 3D Human Pose an...,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/i2l-meshnet-image-to-lixel-prediction-network,Most of the previous image-based 3D human pose and mesh estimation methods e...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/i2l-meshnet-image-to-lixel-prediction-network,False,"[{'term': '3d hand pose estimation', 'scheme': None, 'label': None}, {'term'...",Most of the previous image-based 3D human pose and mesh estimation methods e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Polysemy Deciphering Network for Robust Human-Object Interaction Detection,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/polysemy-deciphering-network-for-robust-human,"To address this issue, in this paper, we propose a novel Polysemy Decipherin...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/polysemy-deciphering-network-for-robust-human,False,"[{'term': 'Human-object interaction detection', 'scheme': None, 'label': Non...","To address this issue, in this paper, we propose a novel Polysemy Decipherin...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cascade Graph Neural Networks for RGB-D Salient Object Detection,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/cascade-graph-neural-networks-for-rgb-d,Current works either simply distill prior knowledge from the corresponding d...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/cascade-graph-neural-networks-for-rgb-d,False,"[{'term': 'Salient object detection', 'scheme': None, 'label': None}]",Current works either simply distill prior knowledge from the corresponding d...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Convolutional Complex Knowledge Graph Embeddings,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/convolutional-complex-knowledge-graph,"In this paper, we study the problem of learning continuous vector representa...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/convolutional-complex-knowledge-graph,False,"[{'term': 'Knowledge graph embeddings', 'scheme': None, 'label': None}, {'te...","In this paper, we study the problem of learning continuous vector representa...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Quantum State Tomography with Conditional Generative Adversarial Networks,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/quantum-state-tomography-with-conditional,We augment a CGAN with custom neural-network layers that enable conversion o...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/quantum-state-tomography-with-conditional,False,"[{'term': 'Quantum state tomography', 'scheme': None, 'label': None}]",We augment a CGAN with custom neural-network layers that enable conversion o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#export

document_store = InMemoryDocumentStore(
    embedding_field="article_emb",
)

In [11]:
#export


model_name = "deepset/sentence_bert"


retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model=model_name,
    use_gpu=use_gpu)

08/12/2020 17:13:10 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
08/12/2020 17:13:10 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/12/2020 17:13:10 - INFO - farm.infer -   Could not find `deepset/sentence_bert` locally. Try to download from model hub ...
08/12/2020 17:13:13 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
08/12/2020 17:13:18 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None


In [12]:
retriever.embedding_model.model

AdaptiveModel(
  (language_model): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
            

In [13]:
feed_df.columns

Index(['title', 'title_detail', 'links', 'link', 'summary', 'summary_detail',
       'id', 'guidislink', 'tags', 'text', 'feed', 'published',
       'published_parsed', 'comments', 'authors', 'author', 'author_detail',
       'updated', 'updated_parsed', 'content', 'href', 'media_thumbnail'],
      dtype='object')

In [14]:
#export

article_texts = feed_df['text']

In [15]:
#export

article_embeddings = retriever.embed_queries(texts=list(article_texts))

Inferencing Samples: 100%|██████████| 74/74 [00:10<00:00,  7.16 Batches/s]


In [16]:
#export

feed_df['article_emb'] = article_embeddings 

In [17]:
#export

document_store.write_documents(feed_df.to_dict(orient='records'))

In [18]:
feed_df.iloc[1]['text']

'To address this issue, in this paper, we propose a novel Polysemy Deciphering Network (PD-Net) that decodes the visual polysemy of verbs for HOI detection in three distinct ways. Code: https://github.com/MuchHair/PD-Net'

In [19]:
def doc_to_dict(doc):
    d = {}
    d['text'] = doc.text
    d['title'] = doc.meta['title']
    d['score'] = doc.query_score
    return d

In [20]:
topic_strings = [
    'computer vision',
    'deep learning',
    'natural language processing'
]

In [21]:
topic = topic_strings[-1]

In [22]:
results = [
    doc_to_dict(doc)
    for doc in retriever.retrieve(
        topic
    )
]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 32.07 Batches/s]


In [23]:
import ktrain

zsl_clf = ktrain.text.ZeroShotClassifier(device='cuda')

In [24]:
scores = [
    zsl_clf.predict(d['text'], topic_strings=topic_strings, max_length=256, include_labels=True)
    for d in results
]

In [25]:
results[0]

{'text': 'Syntactic annotation of corpora in the form of part-of-speech (POS) tags is a key requirement for both linguistic research and subsequent automated natural language processing (NLP) tasks. Code: https://github.com/stheid/SetPOS',
 'title': 'Reliable Part-of-Speech Tagging of Historical Corpora through Set-Valued Prediction',
 'score': 0.5372036650018498}

In [26]:
results_df = pd.DataFrame.from_records(
    {**result, **dict(doc_scores)}
    for (result, doc_scores) in zip(results, scores)
)

In [27]:
results_df

,text,title,score,computer vision,deep learning,natural language processing
0,Syntactic annotation of corpora in the form of part-of-speech (POS) tags is ...,Reliable Part-of-Speech Tagging of Historical Corpora through Set-Valued Pre...,0.537204,0.001010,0.004424,0.944371
1,"In our approach, protected attributes are connected to evaluative words foun...",Discovering and Categorising Language Biases in Reddit,0.453108,0.003528,0.032202,0.454947
2,"Artificial touch would seem well-suited for Reinforcement Learning (RL), sin...",Deep Reinforcement Learning for Tactile Robotics: Learning to Type on a Brai...,0.411201,0.004114,0.022183,0.000430
3,"We propose a novel algorithm, named Open-Edit, which is the first attempt on...",Open-Edit: Open-Domain Image Manipulation with Open-Vocabulary Instructions,0.401655,0.404642,0.097054,0.002871
4,A common task in computational text analyses is to quantify how two corpora ...,Generalized Word Shift Graphs: A Method for Visualizing and Explaining Pairw...,0.386268,0.000250,0.005168,0.076504
5,Neural Language Models and the Turing TestContinue reading on Towards AI — M...,Is GPT-3 ‘Human’?,0.380754,0.000435,0.165146,0.663763
6,"To address this problem, we created a new dataset containing both synthetic ...",Synthetic to Real Unsupervised Domain Adaptation for Single-Stage Artwork Re...,0.364484,0.652031,0.566288,0.001007
7,"For addressing this issue, this paper leverages domain-specific mappings for...",Domain-Specific Mappings for Generative Adversarial Style Transfer,0.359442,0.008340,0.037381,0.002777
8,"To address this issue, in this paper, we propose a novel Polysemy Decipherin...",Polysemy Deciphering Network for Robust Human-Object Interaction Detection,0.346406,0.260132,0.065253,0.032847
9,Hello! Would be grateful if you could share the criteria you use/have encoun...,Any translators or ML specialists here? How does one evaluate a translation ...,0.344971,0.190806,0.059315,0.518846
